# Lab 11: Grid Localization using Bayes Filter (Real Robot)

### <span style="color:rgb(0,150,0)">It is recommended that you close any heavy-duty applications running on your system while working on this lab.</span>

### <span style="color:rgb(0,150,0)">The notebook only provides skeleton code for you to integrate the Localization class with the Real Robot.</span>

<hr>

In [323]:
%load_ext autoreload
%autoreload 2

import traceback
from notebook_utils import *
from Traj import *
import asyncio
import pathlib
import os
from utils import load_config_params
from localization_extras import Localization

# The imports below will only work if you copied the required ble-related python files 
# into the notebooks directory
from ble import get_ble_controller
from base_ble import LOG
from cmd_types import CMD
import numpy as np

# Setup Logger
LOG = get_logger('demo_notebook.log')
LOG.propagate = False

# Init GUI and Commander
gui = GET_GUI()
cmdr = gui.launcher.commander

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2025-04-28 22:37:40,723 | INFO     |: Logger demo_notebook.log already initialized.
2025-04-28 22:37:40,727 | INFO     |: GUI is already running. Shutdown notebook to force restart the GUI.


In [562]:
# Start the plotter
START_PLOTTER()

2025-04-29 01:18:59,233 | INFO     |: Creating New Plotter Process


# The RealRobot class
Define the RealRobot class in the code cell below, based on the documentation and your real robot communication protocol. <br>
This class is used by the **Localization** class to communicate with the real robot. <br>
More specifically, the **Localization** class utilizes the **RealRobot's** member function **perform_observation_loop()** to get the 18 sensor readings and store them in its member variable **obs_range_data**, which is then utilized in the update step.

In [573]:
sensor_index = 0
sensor_data = [[0.0 for _ in range(1)] for _ in range(18)]
yaw_data = [[0.0 for _ in range(1)] for _ in range(18)]
msg = None

def data_collect(uuid, byte_array) :
    global sensor_index
    global sensor_data
    global yaw_data
    msg = ble.bytearray_to_string(byte_array)
    if "," in msg:
        data = msg.split(",")
        if sensor_index < len(sensor_data):
            sensor_data[sensor_index] = [float(data[1])]
            yaw_data[sensor_index] = [float(data[2])]
            sensor_index += 1 

In [574]:
class RealRobot():
    """A class to interact with the real robot
    """
    def __init__(self, commander, ble):
        # Load world config
        self.world_config = os.path.join(str(pathlib.Path(os.getcwd()).parent), "config", "world.yaml")
        
        self.config_params = load_config_params(self.world_config)
        
        # Commander to commuincate with the Plotter process
        # Used by the Localization module to plot odom and belief
        self.cmdr = commander

        # ArtemisBLEController to communicate with the Robot
        self.ble = ble

    def get_pose(self):
        """Get robot pose based on odometry
        
        Returns:
            current_odom -- Odometry Pose (meters, meters, degrees)
        """
        raise NotImplementedError("get_pose is not implemented")

    def perform_observation_loop(self, rot_vel=120):
        """Perform the observation loop behavior on the real robot, where the robot does  
        a 360 degree turn in place while collecting equidistant (in the angular space) sensor
        readings, with the first sensor reading taken at the robot's current heading. 
        The number of sensor readings depends on "observations_count"(=18) defined in world.yaml.
        
        Keyword arguments:
            rot_vel -- (Optional) Angular Velocity for loop (degrees/second)
                        Do not remove this parameter from the function definition, even if you don't use it.
        Returns:
            sensor_ranges   -- A column numpy array of the range values (meters)
            sensor_bearings -- A column numpy array of the bearings at which the sensor readings were taken (degrees)
                               The bearing values are not used in the Localization module, so you may return a empty numpy array
        """
        global sensor_data
        global sensor_index
        global yaw_data

        sensor_index = 0
        sensor_data = [[0.0 for _ in range(1)] for _ in range(18)]
        yaw_data = [[0.0 for _ in range(1)] for _ in range(18)]

        self.ble.start_notify(self.ble.uuid['RX_STRING'], data_collect)
       
        # Send command to rotate + scan
        self.ble.send_command(CMD.START_MAP, "")
       
        import asyncio
        asyncio.run(asyncio.sleep(20))
       
        self.ble.stop_notify(self.ble.uuid['RX_STRING'])
       
        sensor_ranges = np.array(sensor_data) / 1000.0
        sensor_bearings = np.array([])[np.newaxis].T
       
        return sensor_ranges, sensor_bearings


In [575]:
# Get ArtemisBLEController object
ble = get_ble_controller()

# Connect to the Artemis Device
ble.connect()

2025-04-29 01:21:16,256 | INFO     |: Already connected to a BLE device


In [576]:
# Initialize RealRobot with a Commander object to communicate with the plotter process
# and the ArtemisBLEController object to communicate with the real robot
robot = RealRobot(cmdr, ble)

# Initialize mapper
# Requires a VirtualRobot object as a parameter
mapper = Mapper(robot)

# Initialize your BaseLocalization object
# Requires a RealRobot object and a Mapper object as parameters
loc = Localization(robot, mapper)

## Plot Map
cmdr.plot_map()

2025-04-29 01:21:17,295 | INFO     |:  | Number of observations per grid cell: 18
2025-04-29 01:21:17,296 | INFO     |:  | Precaching Views...
2025-04-29 01:21:20,904 | INFO     |:  | Precaching Time: 3.607 secs
2025-04-29 01:21:20,905 | INFO     |: Initializing beliefs with a Uniform Distribution
2025-04-29 01:21:20,906 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107


# Run an update step of the Bayes Filter

In [577]:
# Reset Plots
cmdr.reset_plotter()

# Init Uniform Belief
loc.init_grid_beliefs()

# Get Observation Data by executing a 360 degree rotation motion
loc.get_observation_data()

# Run Update Step
loc.update_step()
loc.plot_update_step_data(plot_data=True)

# Plot Odom and GT
# current_odom, current_gt = robot.get_pose()
# cmdr.plot_gt(0, 0.9144) # top middle (0,3)
cmdr.plot_gt(-0.9144, -0.6096) # bottom left (-3, -2)
# cmdr.plot_gt(1.524, -0.9144) # bottom right (5,-3)
# cmdr.plot_gt(1.524, 0.9144) # top right (5,3)
# cmdr.plot_odom(current_odom[0], current_odom[1])

2025-04-29 01:21:25,075 | INFO     |: Initializing beliefs with a Uniform Distribution
2025-04-29 01:21:25,077 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107
2025-04-29 01:21:45,161 | INFO     |: Update Step
2025-04-29 01:21:45,164 | INFO     |:      | Update Time: 0.002 secs
2025-04-29 01:21:45,165 | INFO     |: Bel index     : (np.int64(2), np.int64(3), np.int64(6)) with prob = 1.0
2025-04-29 01:21:45,167 | INFO     |: Bel_bar prob at index = 0.00051440329218107
2025-04-29 01:21:45,168 | INFO     |: Belief        : (-0.914, -0.305, -50.000)


In [556]:
# cmdr.plot_gt(0, 0.9144) # top middle (0,3)
# cmdr.plot_gt(1.524, -0.9144) # bottom right (5,-3)
# cmdr.plot_gt(1.524, 0.9144) # top right (5,3)
# cmdr.plot_gt(-0.9144, -0.6096) # bottom left (-3, -2)

2025-04-29 01:16:14,092 | INFO     |: Disconnected from AC6D8405-1EA0-1D63-D5D5-053EFC3848E0


In [437]:
print(sensor_data)
print(yaw_data)

[[357.0], [356.0], [417.0], [555.0], [475.0], [728.0], [681.0], [692.0], [908.0], [1462.0], [3218.0], [3419.0], [2233.0], [508.0], [2466.0], [2987.0], [632.0], [589.0]]
[[397.0], [395.0], [431.0], [558.0], [768.0], [2064.0], [2251.0], [654.0], [484.0], [437.0], [2065.0], [1012.0], [523.0], [441.0], [373.0], [381.0], [515.0], [545.0]]


Traceback (most recent call last):
  File "/Users/luluhtutt/cornell/ece4160/sim/FastRobots-sim-release-main/src/plotter.py", line 263, in keyPressEvent
    if event.key() == Qt.Key_Escape:
                      ^^^^^^^^^^^^^
AttributeError: type object 'Qt' has no attribute 'Key_Escape'
Traceback (most recent call last):
  File "/Users/luluhtutt/cornell/ece4160/sim/FastRobots-sim-release-main/src/plotter.py", line 263, in keyPressEvent
    if event.key() == Qt.Key_Escape:
                      ^^^^^^^^^^^^^
AttributeError: type object 'Qt' has no attribute 'Key_Escape'
QGraphicsView::dragLeaveEvent: drag leave received before drag enter


In [554]:
bel_n3n2 = [[-0.610,-0.305,-10.0], [-0.610, -0.305, -30.0]]
bel_03 = [[0.305,0.914,-50.0], [0.305,0.914,-20.0]]
bel_5n3 = [[1.810, -1.210, -40.0], [1.810, -0.914, -50.0]]
bel_53 = [[1.520, 0.910, -30.0], [1.524, 0.610, 150.0]]

In [583]:
print(np.sqrt((-0.9144-(-0.610))**2 + (-0.6096-(-0.305))**2))
print(np.sqrt((0-(0.305))**2 + (0.9144-(0.914))**2))
print(np.sqrt((1.524-(1.810))**2 + (-0.9144-(-1.210))**2))
print(np.sqrt((1.524-(1.524))**2 + (0.9144-(0.610))**2))

0.4306280529645044
0.3050002622949692
0.41130932398864967
0.3044


In [555]:
print(bel_03)

[[0.305, 0.914, -50.0], [0.305, 0.914, -20.0]]
